In [1]:
# start with notebook,
# convert into module later

In [1]:
from pprint import pprint
from util import generate_problem, verify_witness
INF = 1e9

In [3]:
def discrete_graph(N):
    mat = [[INF for _ in range(N)] for _ in range(N)]
    for i in range(N):
        mat[i][i] = 0
    return mat

def generate_d_graph(graph):
    N = len(graph)
    E = discrete_graph(N)
    
    for i in range(N):
        E[i][i] = 0 # redundant

    for i in range(N):
        for j in range(N):
            E[i][j] = graph[i][j]

    for k in range(N):
        for i in range(N):
            for j in range(N):
                E[i][j] = min(
                    E[i][j],
                    E[i][k] + E[k][j],
                )

    return E

def consistent(d_graph):
    for i in range(len(d_graph)):
        if d_graph[i][i] < 0: return False
    return True

def get_min_sol(d_graph):
    return [-d_graph[i][0] for i in range(len(d_graph))]

In [6]:
def pick_constraint_first_unused(available_constraints):
    return list(available_constraints)[0]

def pick_constraint_least_intervals(available_constraints):
    least_intervals = 0
    num = len(available_constraints[least_intervals]['intervals'])
    for idx in range(1, len(available_constraints)):
        if len(available_constraints[idx]['intervals']) < num:
            num = len(available_constraints[idx]['intervals'])
            least_intervals = idx
            
    return available_constraints[least_intervals]

In [7]:
def select_units(available_constraints, graph):
    """
    for each constraint that is simply a single interval - add it
    """

    added = []
    for constr in available_constraints:
        if len(constr['intervals']) == 1:
            i, j, interval = constr['i'], constr['j'], constr['intervals'][0]
            
            graph[i][j] = interval[1]
            graph[j][i] = -interval[0]
            
            added.append(constr)
            
    for a in added:
        available_constraints.remove(a)

def backtrack(constraints, i, graph, stats=None):
    if stats == None:
        stats = {'consistent': 0, 'dead': 0, 'total': 0}
    
    if i == len(constraints):
        stats['total'] += 1
        d_graph = generate_d_graph(graph)
        cons = consistent(d_graph)
        if cons:
            stats['consistent'] += 1
        else:
            stats['dead'] += 1
        return stats, cons
    
    constr = constraints[i]
    
    i, j = constr['i'], constr['j']
    for interval in constr['intervals']:
        stats['total'] += 1
        graph[i][j] = interval[1]
        graph[j][i] = -interval[0]

        d_graph = generate_d_graph(graph)
        if consistent(d_graph):
            x = backtrack(constraints, i+1, graph, stats)
            if x[1]: return stats, True, 0
            elif x[2] < i: return stats, i
        else:
            stats['dead'] += 1
    
        graph[i][j] = INF
        graph[j][i] = INF

    print('i am not sure this line should even get reached (except once at the end)')
    return stats, False, i - 1

def solve(num_variables, constraints, pick_constraint, break_on_sat=True):
    import copy
    T = copy.deepcopy(constraints)
    
    graph = discrete_graph(num_variables)
    select_units(constraints, graph)
    
    return backtrack(
        T, 
        graph,
        pick_constraint,
        list(),
        break_on_sat=break_on_sat,
    )

In [272]:
def backtrack(constraints):
    i = 0
    selection = [None for _ in range(len(constraints))]
    latest = [-1 for _ in range(len(constraints))]
    
    while 0 <= i < len(constraints):
        select_value_gbj(constraints, i, selection, latest)
        print(i)
        if selection[i] is None:
            print('jumping', i, latest[i])
            i = latest[i]
        else:
            i = i + 1
            if i < len(constraints):
                selection[i] = None
                latest[i] = -1
    
    if i == -1:
        print('UNSAT')
        return None
    else:
        # turn selection into minimal working example
        
        graph = discrete_graph(max([max(t['i'], t['j']) for t in constraints]) + 1)
        for k, constr in enumerate(constraints):
            graph[constr['i']][constr['j']] = constr['intervals'][selection[k]][1]
            graph[constr['j']][constr['i']] = -constr['intervals'][selection[k]][0]
        d_graph = generate_d_graph(graph)
        return get_min_sol(d_graph)

def max_k_consistent(constraints, i, selection, latest):
    graph = discrete_graph(max([max(t['i'], t['j']) for t in constraints]) + 1)
    
    # x_i = a
    constr = constraints[i]
    graph[constr['i']][constr['j']] = constr['intervals'][selection[i]][1]
    graph[constr['j']][constr['i']] = -constr['intervals'][selection[i]][0]
    
    k = 0
    while k < i:
        latest[i] = max(latest[i], k)
        
        # a_k-tuple graph
        constr = constraints[k]
        graph[constr['i']][constr['j']] = constr['intervals'][selection[k]][1]
        graph[constr['j']][constr['i']] = -constr['intervals'][selection[k]][0]
        
        d_graph = generate_d_graph(graph)
        if not consistent(d_graph):
            return False
        else:
            k += 1
    
    return True
            
def select_value_gbj(constraints, i, selection, latest):
    intervals = constraints[i]['intervals']
    
    if selection[i] is None:
        selection[i] = -1
    
    while selection[i] + 1 < len(intervals):
        selection[i] += 1
        if max_k_consistent(constraints, i, selection, latest):
            return
    
    selection[i] = None

def solve(constraints):
    from copy import deepcopy
    c2 = deepcopy(constraints)
    c2.sort(key=lambda l: len(l['intervals']))
    return backtrack(c2)

In [288]:
T = generate_problem(16)
T

[{'i': 0, 'j': 0, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 1, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 2, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 3, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 4, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 5, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 6, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 7, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 8, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 9, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 10, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 11, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 12, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 13, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 14, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 15, 'intervals': [(-100, 100)]},
 {'i': 1, 'j': 4, 'intervals': [(-99, -55), (-29, -14), (-13, -7), (63, 93)]},
 {'i': 1,
  'j': 6,
  'intervals': [(-92, -87), (-60, -33), (35, 40), (65, 67), (70, 88)]},
 {'i': 1,
  'j': 8,
  'intervals': [(-57, -28), (-15, -9), (43, 44), (4

In [289]:
wit = solve(T)
wit

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
jumping 30 27
27
28
29
30
jumping 30 27
27
28
29
30
31
32
33
34
35
36
jumping 36 35
35
36
37
38
39
40
jumping 40 32
32
jumping 32 31
31
32
33
jumping 33 32
32
jumping 32 31
31
32
33
jumping 33 31
31
jumping 31 30
30
jumping 30 29
29
30
31
32
33
34
35
36
jumping 36 35
35
36
37
38
39
40
jumping 40 32
32
jumping 32 31
31
32
33
jumping 33 32
32
jumping 32 31
31
32
33
jumping 33 31
31
jumping 31 30
30
jumping 30 29
29
30
31
32
33
34
35
36
jumping 36 35
35
36
37
38
39
40
jumping 40 32
32
jumping 32 31
31
32
33
jumping 33 32
32
jumping 32 31
31
32
33
jumping 33 31
31
jumping 31 30
30
jumping 30 29
29
30
31
32
jumping 32 30
30
jumping 30 29
29
jumping 29 28
28
29
30
31
32
jumping 32 28
28
jumping 28 27
27
jumping 27 26
26
27
28
29
30
jumping 30 27
27
28
29
30
jumping 30 27
27
28
29
30
31
32
jumping 32 28
28
29
30
31
32
jumping 32 28
28
29
30
31
32
jumping 32 28
28
jumping 28 27
27
jumping 27 26
26
27
28
jumping 

KeyboardInterrupt: 

In [290]:
Tpc = PC_1(T)
wit = solve(Tpc)
wit

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
jumping 84 63
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
jumping 91 89
89
90
91
92
93
jumping 93 75
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
jumping 91 89
89
90
91
92
93
jumping 93 92
92
93
94
95
jumping 95 76
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
jumping 91 89
89
90
91
92
93
jumping 93 92
92
jumping 92 91
91
jumping 91 90
90
jumping 90 89
89
jumping 89 88
88
jumping 88 87
87
88
89
90
91
92
93
jumping 93 87
87
jumping 87 86
86
jumping 86 85
85
jumping 85 84
84
85
86
87
88
89
90
91
92
jumping 92 84
84
jumping 84 83
83
84
85
86
87
88
89
90
91
jumping 91 89
89
90
91
92
93
94
95
96
97
98
99
jumping 99 68
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
jumping 86 83
83
84
85
86


KeyboardInterrupt: 

In [287]:
verify_witness(wit, T)

[]

## --- Main Implementation End ---

## --- Preprocessing Implementation ---

Let's test out a few preprocessing algorithms, and below compare how backtrack fares with and without preprocessing

In [268]:
def intersect(I, J):
    """
    intersect intervals given by lists I and J
    note: both lists have to contain ordered disjoint intervals.
    """
    
    # TODO fix this merge lol, experimental
    return cleanup_intervals([
        [max(first[0], second[0]), min(first[1], second[1])] 
          for first in I for second in J  if max(first[0], second[0]) <= min(first[1], second[1])
    ])
    
def compose(I, J):
    """
    compose lists of intervals I and J
    note: make sure they are ordered and disjoint
    """

    return cleanup_intervals([
        (i[0] + j[0], i[1] + j[1])
        for i in I for j in J
    ])

def cleanup_intervals(I):
    """
    given a list of intervals, produce equivalent ordered disjoint intervals
    """
    
    ret = []
    
    for begin, end in sorted(I):
        if ret and ret[-1][1] >= begin - 1:
            ret[-1][1] = max(ret[-1][1], end)
        else:
            ret.append( [begin, end] )
    
    return [(a, b) for a, b in ret]

    
def PC_1(T):
    # i need constraints to be in a specific form to perform random access T_ij
    num_variables = max([max(t['i'], t['j']) for t in T]) + 1
    mat = [ [ None for i in range(num_variables) ] for j in range(num_variables) ]
    for constr in T:
        i, j, intervals = constr['i'], constr['j'], constr['intervals']
        
        mat[i][j] = intervals
        mat[j][i] = [(-b, -a) for a, b in reversed(intervals)]
        # for reverse, we take each interval (a,b) and convert it into (-b,-a)
        # note we want to also reverse the order of all intervals so that it is again increasing
        
    
    # actual PC-1 core algorithm
    
    changed = True
    while changed:
        changed = False
        for k in range(num_variables):
            for i in range(num_variables):
                for j in range(num_variables):
                    # T_ij = T_ij + T_ik x T_kj

                    if mat[i][k] == None or mat[k][j] == None:
                        # if we cannot compose
                        continue

                    elif mat[i][j] == None:
                        # if unset it means no constraint, so only compose
                        mat[i][j] = compose(mat[i][k], mat[k][j])

                    else: # full intersect&compose
                        op = intersect(mat[i][j], compose(mat[i][k], mat[k][j]))
                        if mat[i][j] != op:
                            mat[i][j] = op
                            changed = True

                    if not mat[i][j]:
                        print('PC-1: unsat')
                        return
    
    # convert back to constraint form
    S = []
    for i in range(num_variables):
        for j in range(i, num_variables):
            if mat[i][j]:
                S += [{'i': i, 'j': j, 'intervals': mat[i][j]}]
    return S

## --- Preprocessing End ---

In [65]:
num_variables = 15
T = generate_problem(
    variables=num_variables
)
S = PC_1(T)
S

[{'i': 0, 'j': 0, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 1, 'intervals': [(-94, 100)]},
 {'i': 0, 'j': 2, 'intervals': [(-100, 94)]},
 {'i': 0, 'j': 3, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 4, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 5, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 6, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 7, 'intervals': [(-88, 100)]},
 {'i': 0, 'j': 8, 'intervals': [(-27, 100)]},
 {'i': 0, 'j': 9, 'intervals': [(-77, 100)]},
 {'i': 0, 'j': 10, 'intervals': [(-100, 27)]},
 {'i': 0, 'j': 11, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 12, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 13, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 14, 'intervals': [(-100, 100)]},
 {'i': 1, 'j': 1, 'intervals': [(-6, 6)]},
 {'i': 1, 'j': 2, 'intervals': [(-12, -6)]},
 {'i': 1, 'j': 3, 'intervals': [(-91, 102)]},
 {'i': 1, 'j': 4, 'intervals': [(-75, -9), (-7, 43), (58, 85)]},
 {'i': 1, 'j': 5, 'intervals': [(-114, 132)]},
 {'i': 1, 'j': 6, 'intervals': [(-178, 140)]},
 {'

In [67]:
solve(num_variables, T, pick_constraint=pick_constraint_least_intervals)

42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
6
5
4
4
5
5
4
7
10
9
8
8
7
6
5
4
3
2
1


({'consistent': 1, 'dead': 49, 'total': 106}, True)

In [44]:
len(S)

120

In [270]:
PC_1(T)

[{'i': 0, 'j': 0, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 1, 'intervals': [(-24, 81)]},
 {'i': 0, 'j': 2, 'intervals': [(-89, 16)]},
 {'i': 0, 'j': 3, 'intervals': [(-40, 100)]},
 {'i': 0, 'j': 4, 'intervals': [(-100, 99)]},
 {'i': 0, 'j': 5, 'intervals': [(-56, 69)]},
 {'i': 0, 'j': 6, 'intervals': [(-23, 100)]},
 {'i': 0, 'j': 7, 'intervals': [(-100, 69)]},
 {'i': 0, 'j': 8, 'intervals': [(-5, 100)]},
 {'i': 0, 'j': 9, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 10, 'intervals': [(-96, 55)]},
 {'i': 0, 'j': 11, 'intervals': [(-100, 5)]},
 {'i': 0, 'j': 12, 'intervals': [(-81, 24)]},
 {'i': 0, 'j': 13, 'intervals': [(-5, 100)]},
 {'i': 0, 'j': 14, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 15, 'intervals': [(-100, 52)]},
 {'i': 0, 'j': 16, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 17, 'intervals': [(-96, 67)]},
 {'i': 0, 'j': 18, 'intervals': [(-63, 100)]},
 {'i': 0, 'j': 19, 'intervals': [(-100, 100)]},
 {'i': 1, 'j': 1, 'intervals': [(-24, 24)]},
 {'i': 1, 'j': 2, 'intervals'

In [276]:
len(PC_1(T))

210

In [280]:
PC_1(T)

PC-1: unsat
